In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from anndata import AnnData
import matplotlib as mpl
import matplotlib.pyplot as plt
from tifffile import imread

In [ ]:
def get_boolean_column(spots: str):
    """add column in_correct_celltype.

       in_correct_celltype is TRUE, if gene is in_correct_celltype, else FALSE. 

       Parameters
       ----------
       spots : str
            path to csv-file containing spots-information with columns: gene, x, y, celltype
       
       Returns
       ----------
       pandas DataFrame

    """
    df = pd.read_csv(spots)
    df_2 = df.assign(in_correct_celltype= np.random.choice(["TRUE","FALSE"], len(df.index)))
    return df_2

In [ ]:
#test get_boolean_column
df = get_boolean_column('C:/Users/mdichgan/Documents/Helmholtz/test/spots.csv')
print(df)
print(df['x'].min(), df['x'].max(), df['y'].min(), df['y'].max())
x_min, x_max, y_min, y_max = df['x'].min(), df['x'].max(), df['y'].min(), df['y'].max()


In [ ]:
def plot_spots(x_min: int, x_max: int, y_min: int, y_max: int, spots: str, image: str):
    """plot gene spots
     
       spot is red if entry in 'in_correct_celltype' is FALSE, else blue

       Parameters
       ----------
       x_min: int, x_max: int, y_min: int, y_max: int 
            crop coordinates
       spots: str
            path to csv file containing spots-information with columns: gene, x, y, celltype, in_correct_celltype
       image: str
            path to tif file containing image of dapi stained cell-nuclei
    """
    fig, ax = plt.subplots(figsize=(5,2.7), layout='constrained')
    df = get_boolean_column(spots)
    x = imread(image)
    ax.imshow(x) 
    ax.scatter(df.loc[df['in_correct_celltype']=='FALSE', 'x'], df.loc[df['in_correct_celltype']=='FALSE', 'y'], s = 30, color = 'red', marker ='x')
    ax.scatter(df.loc[df['in_correct_celltype']=='TRUE', 'x'], df.loc[df['in_correct_celltype']=='TRUE', 'y'], s = 10, color = 'blue')
    ax.axis([x_min,x_max],[y_min,y_max])
 

In [ ]:
#test plot_spots
plot_spots(x_min, x_max, y_min, y_max, 'C:/Users/mdichgan/Documents/Helmholtz/test/spots.csv', 'C:/Users/mdichgan/Documents/Helmholtz/test/raw_image.tif')
#TODO: fix 'TypeError: axis() takes from 1 to 2 positional arguments but 3 were given'

In [ ]:
def wrong_spot_density(x_min: int, x_max: int, y_min: int, y_max: int, nr_of_pix: int, smooth: float):
    pass